In [20]:
import numpy as np
import pandas as pd
from os import listdir
import pickle

# Data preparation
This notebook prepares the raw data for further statistical analyses. Participants with extreme behavior are excluded. 

### load data sets

In [21]:
# set path and load file names
path = '../data_empirical_raw/'
fileNames = listdir(path)
fileNames = [file for file in fileNames if not file.startswith('meta_') ] # don't load the meta files
print('Number of participants: {}'.format(len(fileNames)))

Number of participants: 74


## Extract the important information from the raw data files
The resulting data includes: participant ID, block no., trial no., time out, used sequence of actions, reaction time, reward

In [22]:
# this functions loads and prepares data from one participant and returns a panda
def prepare_data_participant(fname, path):
    
    # define columns to use of the raw file
    cols = ['sender', 'sender_id', 'move', 'blockN', 'trialN', 'duration','reward', 'moveCount', 'movesTrial'] 
    # deleted: 'visitedFields', 'points', 'response', 'diff',
    
    # load data from current participant
    df = pd.read_csv(path+fname, header=0, usecols=cols) # read data from csv
    
    # filter data for relevant rows
    filterExperiment = df['sender_id'].str[0:4]=='0_12' # just experiment (not training as well)
    filterMoves = df['sender']=='Move' # only moves
    filterFeedback = df['sender']=='Feedback' # only feedback with difference and reward (incl bonus)
    
    # get df with just the feedback rows
    dfTrials = df.loc[(filterExperiment) & (filterFeedback), 'trialN':'reward'].reset_index()

    # get RT for the whole trial 
    dfRTs = df['duration'].loc[(filterExperiment) & (filterMoves)].groupby([df['blockN'], df['trialN']], sort=False).sum().rename('RT').reset_index()
 
    dfTrials['RT'] = dfRTs['RT']
    dfTrials['file_name'] = fname

    # delete useless index column
    del dfTrials['index']

    return dfTrials 

In [23]:
df = pd.DataFrame()

# loop over all participants
for idx, fileName in enumerate(fileNames):
    dfParticipant = prepare_data_participant(fileName, path) # get prepared data
    dfParticipant['Participant_ID'] = idx # add participant ID
    df = pd.concat([df, dfParticipant], ignore_index=True) # concatenate to one big panda

## Define time out trials and add action sequence IDs

In [24]:
# add indicator column for time out trials (not all four moves completed)
df['time_out'] = 0
df.loc[df['moveCount']<4, 'time_out'] = 1

In [25]:
# load file with the sequences (IDs)
f = open('../data_task/sequences.pkl', 'rb')
sequences = pickle.load(f)
f.close()

In [26]:
# convert the moves from string into integer (excl. time out trials)
df.loc[df['time_out']==0, 'movesTrial'] = df.loc[df['time_out']==0, 'movesTrial'].apply(lambda x: [int(val) for val in x.split(',') ])

In [27]:
# add the sequence IDs
df['sequence_ID'] = np.nan
df.loc[df['time_out']==0, 'sequence_ID'] = df.loc[df['time_out']==0, 'movesTrial'].apply(lambda row: np.where((sequences == row).all(axis=1))[0][0])

### Calculate DAS proportions to exclude participants with extreme high

In [28]:
idxDAS = 4 # set the DAS index

In [29]:
# extract the sequences of actions for all valid trials
dfSequences = df.loc[df['time_out']==0, ['Participant_ID', 'sequence_ID']]

# count how often each participant has used each sequence of
dfSequences = dfSequences.groupby('Participant_ID').value_counts().rename('count_sequence').reset_index()

In [30]:
# calculate the proportion of DAS choices
nTrialsParticipants = dfSequences['count_sequence'].groupby(dfSequences['Participant_ID']).sum().to_numpy()
nDASParticipants = dfSequences.loc[dfSequences['sequence_ID']==idxDAS, 'count_sequence'].to_numpy()
pDASParticipants = nDASParticipants/nTrialsParticipants

Find participants with $p(\text{DAS})>90%$

In [31]:
pDAS90 = np.where(pDASParticipants<.90)

print $p(\text{DAS})$ of the outliers

In [32]:
pDASParticipants[np.where(pDASParticipants>.90)].round(2)

array([0.99, 1.  , 0.94, 0.91])

Exclude outliers from the panda

In [34]:
df = df.loc[df['Participant_ID'].isin(pDAS90[0])]

### create a new participant ID column without missing IDs

In [35]:
# remove old ID columns and add new one
nTrials = len(df['trialN'].unique())*len(df['blockN'].unique())
newIDColumn = np.arange(len(df['Participant_ID'].unique())).repeat(nTrials)
df.pop('Participant_ID')
df.insert(0, 'Participant_ID', newIDColumn) # insert new ID column at first position

### save the resulting panda for further analyses

In [36]:
df.to_csv('../data_empirical/data_cleaned.csv', index=False)

### create df that stores the file names and corresponding IDs of main df

In [37]:
dfIDs = df[['Participant_ID', 'file_name']].groupby('file_name').nth(1).reset_index()
dfIDs.pop('index')
dfIDs

,Participant_ID,file_name
0,0,1635170782.csv
1,1,1635086955.csv
2,2,1635587894.csv
3,3,1635169562.csv
4,4,1635250159.csv
...,...,...
65,65,1635702814.csv
66,66,1635431069.csv
67,67,1635254246.csv
68,68,1635198780.csv


In [38]:
dfIDs.to_csv('../data_empirical/data_ID_filename.csv', index=False)

### create df with demographics

In [39]:
# load file with demographic data
dfDemographics = pd.read_csv('../data_empirical/demographic_data.csv', sep=',')

In [40]:
dfDemographics # Record_ID was created by RedCap

,Record_ID,Age,Gender,file_name
0,1,19,weiblich,1634746180.csv
1,3,35,weiblich,1634753718.csv
2,5,25,weiblich,1634893289.csv
3,6,24,weiblich,1635170782.csv
4,7,27,weiblich,1634933833.csv
...,...,...,...,...
69,131,23,weiblich,1636031635.csv
70,133,18,männlich,1636044587.csv
71,134,26,männlich,1636070864.csv
72,136,31,weiblich,1636152235.csv


In [41]:
# delete outliers
dfDemographics = dfDemographics.loc[dfDemographics['file_name'].isin(df['file_name'].unique())]

# add IDs from main df
dfDemographics = pd.merge(dfDemographics, dfIDs, on='file_name')

In [44]:
# save
dfDemographics.to_csv('../data_empirical/demographic_data_IDs.csv', index=False)

### Calculate Demographics Stats

In [45]:
# calculate mean age and sd
dfDemographics['Age'].mean().round(2)

24.06

In [46]:
dfDemographics['Age'].std() #.round(2)

4.627848012378202

In [47]:
dfDemographics['Gender'].value_counts()

Gender
weiblich    50
männlich    20
Name: count, dtype: int64

## Look at answers about strategy, and problems

In [28]:
# this functions loads and prepares data from one participant and returns a panda
def prepare_data_participant_answers(fname, path):
    
    # define columns to use of the raw file
    cols = ['sender', 'Strategie', 'Zweck', 'Probleme']

    dfAnswers = pd.DataFrame()
    
    # load data from current participant
    # check if participant filled out form
    try:
        df = pd.read_csv(path+fname, header=0, usecols=cols) # read data from csv
    
    except ValueError:
        print(f'no data for {fname}')
    
    else:
        # filter data for relevant rows
        filterAnswers = df['sender']=='Form'
       
        # get df with just the feedback rows
        dfAnswers = df.loc[filterAnswers].reset_index()

        # delete useless index column
        del dfAnswers['index']

    finally:
        return dfAnswers 

In [29]:
dfAnswers = pd.DataFrame()

# loop over all participants
for idx, fileName in enumerate(fileNames):
    dfParticipant = prepare_data_participant_answers(fileName, path) # get prepared data
    dfParticipant['Participant_ID'] = idx # add participant ID
    dfAnswers = pd.concat([dfAnswers, dfParticipant], ignore_index=True) # concatenate to one big panda

no data for 1635351725.csv


In [42]:
dfAnswers

,sender,Strategie,Zweck,Probleme,Participant_ID
0,Form,Folgen der vorgegebenen Strecke bis zu gewisse...,"Risikobereitschaft, Entscheidungsverlässlichke...",nein\n,0
1,Form,Ab etwa 15 Punkten Abweichung habe ich eine an...,"Möglicherweise soll untersucht werden, ab welc...",Ich habe eher intuitiv geklickt ohne genau zu ...,1
2,Form,NaN,NaN,NaN,2
3,Form,Wenn der vorgeschlagene Weg eine ähnliche Punk...,"Test, ob man eigene Wege versucht?",Nein,3
4,Form,Im Zweifel die Route mit bekanntem Erwartungswert,NaN,zum Teil habe ich mich mit der Wertigkeit der ...,4
...,...,...,...,...,...
68,Form,Zuerst immer die Standardabfolge und währendde...,Nein.,Nein.,69
69,Form,Wenn der Zielwert einstellig war (positiv oder...,"Dadurch, dass man gesagt bekommt und auch selb...",Ich hab mich manchmal schwer getan von dem Sta...,70
70,Form,Ich habe meistens die vorgegebene graue Strate...,"Ich könnte mir vorstellen, dass erforscht werd...",Nein.,71
71,Form,Bei Zielen mit einer geringen Zahl immer das S...,nein,nein,72


In [48]:
# save it as csv
dfAnswers.to_csv('../data_empirical/data_free_answers.csv', sep=';', encoding='utf-8-sig')

In [30]:
# print date of last changes and version numbers
%load_ext watermark

%watermark -n -u -v -iv -w

Last updated: Fri Jan 03 2025

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.20.0

numpy : 1.25.2
pandas: 2.2.1

Watermark: 2.3.1

